In [1]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

#import TropiGAT_functions 
#from TropiGAT_functions import get_top_n_kltypes ,clean_print 

import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool
warnings.filterwarnings("ignore")

# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
path_ensemble = f"{path_work}/ficheros_28032023/ensemble_2809"

In [2]:
import TropiGAT_functions 

> Make model : 

In [3]:
dico_models = TropiGAT_functions.make_ensemble_TropiGAT(path_ensemble)

> Ferriol

In [4]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
path_Dpo_domain_org = "/media/concha-eloko/Linux/depolymerase_building/clean_77_phages_depo"

dpo_embeddings = pd.read_csv(f"{path_project}/rbp_work/Dpo_domains_77.esm2.embedding.csv", sep = "," , header = None)
dpo_embeddings = dpo_embeddings.drop([1281] , axis = 1)
dpo_embeddings.set_index([0], inplace = True)

In [5]:
dpo_embeddings

,1,2,3,4,5,6,7,8,9,10,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,,,,,,,,,,,,,,,,,,,,,
K15PH90__cds_55_Dpo_domain,-0.028760,0.046677,-0.010773,0.028452,-0.090442,0.027041,0.004249,-0.083708,0.022172,0.090119,...,-0.032166,-0.012386,0.079159,0.012298,0.027317,0.037254,0.069599,-0.097522,0.067495,0.062502
K80PH1317b__cds_54_Dpo_domain,0.007689,0.036850,-0.006928,-0.056424,-0.090723,0.018707,0.014913,-0.070090,0.073792,0.055322,...,0.017004,-0.000657,0.059184,-0.006782,0.023955,0.035585,0.048035,-0.081247,0.043776,0.118674
K64PH164C4__cds_24_Dpo_domain,0.015762,0.062429,-0.003427,-0.003609,-0.101109,0.028121,0.004342,-0.096114,0.062562,0.027864,...,0.042829,0.029579,0.095159,0.024894,0.002837,0.046701,0.062497,-0.084956,0.027426,0.051051
K5lambda5__cds_196_Dpo_domain,0.040111,0.046436,-0.012045,-0.043877,-0.100054,-0.028328,0.028640,-0.047144,0.065727,0.047312,...,0.017347,0.008084,0.096149,-0.031008,0.040423,0.082593,0.050161,-0.105612,0.023642,0.081104
K11PH164C1__cds_46_Dpo_domain,0.017319,0.077582,-0.001212,-0.030026,-0.070916,-0.011639,0.006673,-0.078486,0.072836,0.046921,...,0.005777,-0.032808,0.099658,-0.028466,0.020794,0.082009,0.030658,-0.091195,0.047744,0.105303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
K61PH164C1__cds_10_Dpo_domain,0.038221,0.054935,0.010196,-0.022653,-0.098234,0.015615,0.016274,-0.050646,0.050069,0.042166,...,0.005035,0.002913,0.103508,-0.006524,0.026563,0.044091,0.034147,-0.095066,0.054782,0.054492
K60PH164C1__cds_96_Dpo_domain,0.031801,0.045489,-0.020759,-0.023341,-0.087468,0.019555,-0.035902,-0.042261,0.063424,0.040513,...,0.017682,0.015435,0.078165,0.023697,0.016046,0.060408,0.054158,-0.090769,0.066702,0.084330
K63PH128__cds_22_Dpo_domain,0.012132,0.044426,-0.010644,-0.017182,-0.043079,-0.016834,-0.010763,-0.082545,0.046603,0.070069,...,0.050513,0.032082,0.108994,0.016834,-0.003678,0.051204,0.083290,-0.103134,0.023146,0.039394


> Bea

In [6]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

bea_embeddings = pd.read_csv(f"{path_project}/Bea_phages.esm2.embedding.csv", sep = "," , header = None)
bea_embeddings = bea_embeddings.drop([1281] , axis = 1)
bea_embeddings.set_index([0], inplace = True)
bea_embeddings


,1,2,3,4,5,6,7,8,9,10,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,,,,,,,,,,,,,,,,,,,,,
A1c_00034,-0.031228,0.018990,0.011914,-0.036915,-0.085279,0.074510,0.038491,-0.092385,0.041953,0.006984,...,0.024149,-0.001948,0.086334,0.015169,-0.009941,0.070748,0.061202,-0.088183,0.029006,0.092003
A1h_00013,0.003407,0.058613,0.004211,-0.032270,-0.080894,0.003224,0.026575,-0.049651,0.068412,0.048845,...,-0.037309,-0.002876,0.103554,-0.011835,0.040593,0.037963,0.031057,-0.102158,0.077059,0.088456
S11a_00036,-0.000087,0.046707,0.027543,-0.037515,-0.073041,0.012802,0.033908,-0.055600,0.117382,0.031282,...,-0.037399,0.023287,0.144737,-0.022651,0.048932,0.046084,0.025735,-0.100239,0.049737,0.135533
A1q_00023,0.035220,-0.004737,-0.047403,0.058447,-0.127027,0.047977,0.093704,-0.168074,0.073823,-0.002389,...,0.015234,0.019885,0.038854,0.031011,-0.032295,-0.027215,0.027464,-0.042393,-0.133780,0.124112
A1q_00010,0.006108,0.036785,0.033718,-0.061564,-0.102759,0.008589,0.025191,-0.038584,0.046164,0.036677,...,-0.030083,-0.019619,0.079121,-0.012699,0.048490,0.054487,0.050371,-0.102361,0.034225,0.098482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A1e_00012,0.023357,0.039636,-0.014646,-0.014371,-0.082256,0.020024,-0.017925,-0.063363,0.068452,0.040234,...,0.016703,-0.005695,0.106709,-0.003896,0.034324,0.069450,0.058052,-0.072267,0.086228,0.071590
M5a_00170,-0.021793,0.077967,-0.017637,-0.069029,-0.085250,0.012727,0.054289,-0.040869,0.069873,0.026029,...,-0.023246,-0.027288,0.092628,-0.031385,0.033944,0.073543,0.025255,-0.100661,0.049953,0.117564
A1j_00002,0.036885,-0.005403,-0.047889,0.057499,-0.128569,0.041746,0.098268,-0.166326,0.072257,-0.004946,...,0.011941,0.016783,0.038163,0.026237,-0.031345,-0.027221,0.030271,-0.040723,-0.132024,0.123907


> Towndsend

In [7]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

towndsend_embeddings = pd.read_csv(f"{path_project}/Townsed_phages.esm2.embedding.csv", sep = "," , header = None)
towndsend_embeddings = towndsend_embeddings.drop([1281] , axis = 1)
towndsend_embeddings.set_index([0], inplace = True)

towndsend_embeddings

,1,2,3,4,5,6,7,8,9,10,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,,,,,,,,,,,,,,,,,,,,,
PP187_gp233,0.016811,0.073770,-0.009818,-0.036033,-0.085370,-0.016310,0.007661,-0.083562,0.072318,0.050186,...,-0.004649,-0.006715,0.121524,-0.004321,0.019053,0.074084,0.042733,-0.064831,0.040088,0.062354
LCALLHIG_00050,0.015476,0.014564,0.016806,-0.042899,-0.071834,0.021123,0.059287,-0.088285,0.081127,0.022274,...,0.016373,-0.006337,0.073145,-0.024415,0.028782,0.048048,0.012546,-0.085356,-0.006697,0.106924
NBNDMPCG_00165,0.097851,-0.005817,0.044202,0.066516,-0.062639,0.012461,-0.056078,-0.046124,0.025547,0.090553,...,0.091665,0.004563,0.011115,0.032996,0.009146,0.084371,0.129982,-0.089394,-0.080854,-0.026717
EKPIEFBL_00177,0.062666,-0.010341,-0.000893,0.034539,-0.107698,0.021799,-0.030059,-0.039033,-0.016901,0.012905,...,0.085217,-0.028300,0.048211,0.066592,0.019395,0.056614,0.101654,-0.031980,-0.056580,0.012406
NBNDMPCG_00161,0.003200,0.046110,0.004314,-0.063110,-0.094585,-0.018906,-0.011139,-0.041858,0.066822,0.052831,...,-0.010727,-0.004597,0.068099,-0.013416,0.020944,0.050755,0.065946,-0.076776,0.066887,0.054853
NBNDMPCG_00168,0.001259,0.060305,0.032569,-0.041890,-0.089129,-0.013875,0.007752,-0.087015,0.005353,0.029473,...,0.015444,-0.034203,0.108186,-0.001604,0.013760,0.058938,0.048440,-0.139902,0.032722,0.077596
PP187_gp235,-0.002062,0.030683,0.003746,-0.050332,-0.096769,-0.008238,-0.007483,-0.037646,0.022372,0.045324,...,0.000123,0.011784,0.091126,-0.005352,0.009611,0.032730,0.073179,-0.076450,0.046952,0.091653
PP187_gp225,0.056159,0.023400,0.017573,0.018860,-0.138268,0.037129,-0.049114,-0.078060,0.031631,0.034067,...,0.026249,-0.017010,0.071993,0.054659,0.010754,0.058265,0.042473,-0.050659,0.040769,0.031806
EONHMLJF_00087,0.040388,0.038805,0.016201,0.074880,-0.099272,0.002561,0.038002,-0.066288,0.094638,0.019382,...,0.003874,-0.040858,0.037478,0.061814,0.134470,-0.062697,0.019345,-0.057822,0.080840,0.092654


In [9]:
towndsend_embeddings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, PP187_gp233 to PP187_gp237
Columns: 1280 entries, 1 to 1280
dtypes: float64(1280)
memory usage: 440.3+ KB


>Others 

In [7]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

others_embeddings = pd.read_csv(f"{path_project}/Others_phages.esm2.embedding.csv", sep = "," , header = None)
others_embeddings = others_embeddings.drop([1281] , axis = 1)
others_embeddings.set_index([0], inplace = True)

namesother_df = pd.read_csv(f"{path_project}/Others/index_others.tsv", sep = "\t" , names = ["index_phage", "index_prot","prot_name"])
new_index = []
for _,index in enumerate(others_embeddings.index) : 
    i_phage = int(index.split("__")[0])
    i_prot = int(index.split("__")[1])
    prot = namesother_df[(namesother_df["index_phage"] == i_phage) & (namesother_df["index_prot"] == i_prot)]["prot_name"].values[0]
    new_index.append(prot)

others_embeddings.index = new_index
others_embeddings.index.name = 0

In [11]:
others_embeddings 

,1,2,3,4,5,6,7,8,9,10,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
0,,,,,,,,,,,,,,,,,,,,,
NC_025418.1_prot_YP_009098385.1_34,-0.034347,0.023348,0.015016,-0.027898,-0.080356,0.076239,0.042055,-0.095709,0.035269,0.006119,...,0.023533,0.001364,0.077270,0.012855,-0.008513,0.072102,0.049917,-0.085625,0.019097,0.094144
NC_029099.1_prot_YP_009226011.1_50,0.018297,0.040493,-0.013773,-0.007963,-0.070584,0.001886,0.004620,-0.087347,0.072146,0.076061,...,0.042868,0.037848,0.101484,0.018096,0.002869,0.033552,0.075552,-0.083361,0.025799,0.028155
MT966873.1_prot_QOV05502.1_43,0.018386,0.014588,0.021574,-0.000521,-0.077781,0.012065,0.005141,-0.071457,0.089763,0.041767,...,0.035410,0.030719,0.098420,0.033683,0.009985,0.013877,0.034163,-0.059480,0.022797,0.041875
NC_013649.2_prot_YP_003347651.1_57,0.010830,0.031390,-0.007675,-0.040679,-0.057371,-0.005628,0.003471,-0.094184,0.047946,0.046877,...,0.026264,0.039920,0.113341,-0.002912,-0.004148,0.048235,0.084940,-0.083238,0.019559,0.044707
AB716666.1_prot_BAP15736.1_24,0.043144,-0.009100,-0.034960,0.056230,-0.123851,0.046627,0.097184,-0.159881,0.081871,-0.003064,...,0.016950,0.018673,0.025980,0.035958,-0.037824,-0.027213,0.028701,-0.037570,-0.121534,0.118953
MK903728.1_prot_QDF14645.1_43,0.023176,0.010373,0.012069,-0.008828,-0.075488,0.001540,-0.002295,-0.066814,0.086006,0.042304,...,0.036544,0.030662,0.093922,0.021278,0.010939,0.000706,0.029708,-0.062703,0.016923,0.052734
NC_031246.1_prot_YP_009302756.1_52,-0.030381,0.025787,0.009855,-0.031848,-0.084225,0.081263,0.045987,-0.102523,0.044074,-0.001147,...,0.022237,0.001937,0.088900,0.011215,-0.007063,0.085473,0.050315,-0.074520,0.029399,0.106660
MZ826764.1_prot_UCR74083.1_31,0.018840,0.072161,0.013396,0.003150,-0.151668,0.041043,-0.018544,-0.054340,0.067439,-0.004516,...,0.064134,0.003406,0.102453,0.042068,0.029047,0.136823,0.026999,-0.069048,0.046625,0.030168
MW655991.1_prot_QUU29414.1_2,0.045209,0.080681,-0.008443,-0.019103,-0.084513,0.013202,0.013670,-0.036699,0.083001,0.002771,...,0.014410,0.026337,0.090563,0.009828,0.017021,0.070165,0.015899,-0.063195,0.028067,0.061598


***
> run the predictions 

In [8]:
# Run the predictions ferriol :
ferriol_predictions = {}
for dpo in dpo_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([dpo_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    ferriol_predictions[dpo] = pred

In [9]:
ferriol_predictions

{'K15PH90__cds_55_Dpo_domain': {'KL145': 0.7034,
  'KL48': 0.9215,
  'KL7': 0.9829,
  'KL118': 0.9609,
  'KL15': 0.9745,
  'KL3': 0.9287,
  'KL43': 0.8158,
  'KL136': 0.9763,
  'KL18': 0.6678,
  'KL149': 0.5068},
 'K80PH1317b__cds_54_Dpo_domain': {'KL128': 0.756,
  'KL74': 0.9412,
  'KL9': 0.6034,
  'KL48': 0.5076,
  'KL81': 0.8484,
  'KL110': 0.5457,
  'KL149': 0.5951},
 'K64PH164C4__cds_24_Dpo_domain': {'KL145': 0.5135,
  'KL13': 0.9587,
  'KL14': 0.8466,
  'KL64': 0.9529,
  'KL3': 0.8553,
  'KL123': 0.8029,
  'KL110': 0.722,
  'KL21': 0.7637},
 'K5lambda5__cds_196_Dpo_domain': {'KL145': 0.9503,
  'KL74': 0.817,
  'KL13': 0.5666,
  'KL48': 0.8594,
  'KL52': 0.5992,
  'KL45': 0.5474,
  'KL12': 0.9675,
  'KL118': 0.5642,
  'KL2': 0.6205,
  'KL53': 0.8357,
  'KL46': 0.5444,
  'KL127': 0.7403,
  'KL3': 0.6205,
  'KL136': 0.8242,
  'KL110': 0.6704,
  'KL18': 0.976,
  'KL149': 0.6745},
 'K11PH164C1__cds_46_Dpo_domain': {'KL145': 0.9378,
  'KL74': 0.9436,
  'KL45': 0.6045,
  'KL57': 0.5124,

In [7]:
# format the results : 
ferriol_pred_formated = TropiGAT_functions.format_predictions(ferriol_predictions , sep = "__")
TropiGAT_functions.clean_print(ferriol_pred_formated)

{'K10PH82C1': {'KL10': 0.5707,
               'KL110': 0.9517,
               'KL118': 0.5704,
               'KL12': 0.9234,
               'KL127': 0.8499,
               'KL128': 0.9694,
               'KL136': 0.7698,
               'KL2': 0.6289,
               'KL25': 0.6036,
               'KL3': 0.8678,
               'KL34': 0.9265,
               'KL45': 0.7,
               'KL46': 0.7351,
               'KL48': 0.8131,
               'KL53': 0.9169,
               'KL60': 0.9817,
               'KL62': 0.9791,
               'KL63': 0.5017,
               'KL74': 0.796},
 'K11PH164C1': {'KL105': 0.5814,
                'KL111': 0.7759,
                'KL127': 0.9122,
                'KL145': 0.9378,
                'KL24': 0.5383,
                'KL3': 0.9619,
                'KL39': 0.5719,
                'KL45': 0.6045,
                'KL46': 0.7452,
                'KL57': 0.5124,
                'KL60': 0.8816,
                'KL62': 0.8212,
                'KL64': 

***

In [10]:
# Run the predictions Bea :
bea_predictions = {}
for dpo in bea_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([bea_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    bea_predictions[dpo] = pred

In [13]:
# format the results : 
bea_pred_formated = TropiGAT_functions.format_predictions(bea_predictions , sep = "_")
TropiGAT_functions.clean_print(bea_pred_formated)

{'A1a': {'KL110': 0.9635,
         'KL123': 0.9964,
         'KL125': 0.7587,
         'KL127': 0.6217,
         'KL14': 0.5576,
         'KL145': 0.6561,
         'KL18': 0.5223,
         'KL2': 0.5835,
         'KL28': 0.5783,
         'KL29': 0.8226,
         'KL3': 0.9582,
         'KL39': 0.5847,
         'KL57': 0.7262,
         'KL7': 0.9813,
         'KL70': 0.9127,
         'KL74': 0.9737,
         'KL8': 0.8445,
         'KL9': 0.9738},
 'A1b': {'KL1': 0.646,
         'KL110': 0.9806,
         'KL114': 0.6447,
         'KL12': 0.7789,
         'KL122': 0.7035,
         'KL123': 0.9989,
         'KL125': 0.9129,
         'KL127': 0.5555,
         'KL128': 0.8204,
         'KL13': 0.8513,
         'KL136': 0.5726,
         'KL14': 0.6044,
         'KL145': 0.8987,
         'KL18': 0.64,
         'KL28': 0.5022,
         'KL29': 0.5146,
         'KL3': 0.9032,
         'KL34': 0.8648,
         'KL39': 0.5375,
         'KL43': 0.6345,
         'KL48': 0.6458,
         'KL5': 0.59

In [44]:
towndsend_embeddings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, PP187_gp233 to PP187_gp237
Columns: 1280 entries, 1 to 1280
dtypes: float64(1280)
memory usage: 441.4+ KB


In [11]:
# Run the predictions Towndsend :
towndsend_predictions = {}
for dpo in towndsend_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([towndsend_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    towndsend_predictions[dpo] = pred

In [12]:
towndsend_predictions

{'PP187_gp233': {'KL145': 0.92,
  'KL13': 0.5693,
  'KL9': 0.5912,
  'KL48': 0.8594,
  'KL52': 0.644,
  'KL151': 0.9322,
  'KL45': 0.8479,
  'KL12': 0.9736,
  'KL53': 0.9801,
  'KL46': 0.7238,
  'KL62': 0.651,
  'KL127': 0.541,
  'KL3': 0.9868,
  'KL25': 0.5428,
  'KL123': 0.8083,
  'KL60': 0.9681,
  'KL136': 0.6111,
  'KL18': 0.9487},
 'LCALLHIG_00050': {'KL111': 0.8015,
  'KL74': 0.6607,
  'KL14': 0.5915,
  'KL8': 0.5729,
  'KL7': 0.9614,
  'KL62': 0.887,
  'KL169': 0.8966,
  'KL3': 0.9994,
  'KL149': 0.6911},
 'NBNDMPCG_00165': {'KL128': 0.5339,
  'KL74': 0.9449,
  'KL13': 0.9183,
  'KL9': 0.9018,
  'KL28': 0.7624,
  'KL151': 0.7662,
  'KL8': 0.7132,
  'KL125': 0.762,
  'KL29': 0.6806,
  'KL116': 0.984,
  'KL127': 0.6338,
  'KL3': 0.9998,
  'KL123': 0.9973,
  'KL60': 0.7599,
  'KL81': 0.8811,
  'KL5': 0.557,
  'KL110': 0.6492,
  'KL106': 0.841,
  'KL149': 0.9166},
 'EKPIEFBL_00177': {'KL145': 0.9838,
  'KL10': 0.5472,
  'KL22': 0.7259,
  'KL48': 0.7599,
  'KL14': 0.7783,
  'KL16': 0

In [35]:
# format the results : 
towndsend_pred_formated = TropiGAT_functions.format_predictions(towndsend_predictions , sep = "_")
TropiGAT_functions.clean_print(towndsend_pred_formated)

{'BLCJPOBP': {'KL110': 0.9665,
              'KL123': 0.9974,
              'KL125': 0.8095,
              'KL127': 0.5507,
              'KL14': 0.6069,
              'KL145': 0.6233,
              'KL3': 0.944,
              'KL57': 0.6884,
              'KL7': 0.9892,
              'KL70': 0.8341,
              'KL74': 0.8679,
              'KL8': 0.905,
              'KL9': 0.9512},
 'DIMCIIMF': {'KL10': 0.9462,
              'KL105': 0.9927,
              'KL106': 0.992,
              'KL108': 0.9986,
              'KL110': 0.9494,
              'KL114': 0.9831,
              'KL116': 0.9865,
              'KL118': 0.9904,
              'KL12': 0.7573,
              'KL123': 0.9984,
              'KL125': 0.9881,
              'KL127': 0.9494,
              'KL128': 0.8972,
              'KL13': 0.817,
              'KL136': 0.7737,
              'KL14': 0.5173,
              'KL145': 0.863,
              'KL149': 0.9996,
              'KL15': 0.7064,
              'KL16': 1.0,
  

In [43]:
others_embeddings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, NC_025418.1_prot_YP_009098385.1_34 to MT966872.1_prot_QOV05454.1_38
Columns: 1280 entries, 1 to 1280
dtypes: float64(1280)
memory usage: 371.3+ KB


In [20]:
others_embeddings.index

Index(['NC_025418.1_prot_YP_009098385.1_34',
       'NC_029099.1_prot_YP_009226011.1_50', 'MT966873.1_prot_QOV05502.1_43',
       'NC_013649.2_prot_YP_003347651.1_57', 'AB716666.1_prot_BAP15736.1_24',
       'MK903728.1_prot_QDF14645.1_43', 'NC_031246.1_prot_YP_009302756.1_52',
       'MZ826764.1_prot_UCR74083.1_31', 'MW655991.1_prot_QUU29414.1_2',
       'AB897757.1_prot_BAQ02839.1_59', 'NC_031246.1_prot_YP_009302745.1_41',
       'AB897757.1_prot_BAQ02841.1_61', 'MZ826764.1_prot_UCR74085.1_33',
       'NC_025418.1_prot_YP_009098375.1_24', 'AB897757.1_prot_BAQ02844.1_64',
       'MK903728.1_prot_QDF14639.1_37', 'AB897757.1_prot_BAQ02843.1_63',
       'AB897757.1_prot_BAQ02838.1_58', 'MT542697.1_prot_QKY78353.1_44',
       'MZ826764.1_prot_UCR74084.1_32', 'ON146449.1_prot_UPW35150.1_13',
       'MK903728.1_prot_QDF14644.1_42', 'AB716666.1_prot_BAP15746.1_34',
       'MT542697.1_prot_QKY78347.1_38', 'MZ826764.1_prot_UCR74082.1_30',
       'MT966873.1_prot_QOV05496.1_37', 'MN781108.1_pro

In [23]:
# Run the predictions Others :
other_predictions = {}
for index, dpo in enumerate(others_embeddings.index) :
    if dpo not in ["MN781108.1_prot_QGZ15323.1_262"] : 
        graph_dpo = TropiGAT_functions.make_query_graph([others_embeddings.loc[dpo].values])
        pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
        other_predictions[dpo] = pred

NC_025418.1_prot_YP_009098385.1_34
NC_029099.1_prot_YP_009226011.1_50
MT966873.1_prot_QOV05502.1_43
NC_013649.2_prot_YP_003347651.1_57
AB716666.1_prot_BAP15736.1_24
MK903728.1_prot_QDF14645.1_43
NC_031246.1_prot_YP_009302756.1_52
MZ826764.1_prot_UCR74083.1_31
MW655991.1_prot_QUU29414.1_2
AB897757.1_prot_BAQ02839.1_59
NC_031246.1_prot_YP_009302745.1_41
AB897757.1_prot_BAQ02841.1_61
MZ826764.1_prot_UCR74085.1_33
NC_025418.1_prot_YP_009098375.1_24
AB897757.1_prot_BAQ02844.1_64
MK903728.1_prot_QDF14639.1_37
AB897757.1_prot_BAQ02843.1_63
AB897757.1_prot_BAQ02838.1_58
MT542697.1_prot_QKY78353.1_44
MZ826764.1_prot_UCR74084.1_32
ON146449.1_prot_UPW35150.1_13
MK903728.1_prot_QDF14644.1_42
AB716666.1_prot_BAP15746.1_34
MT542697.1_prot_QKY78347.1_38
MZ826764.1_prot_UCR74082.1_30
MT966873.1_prot_QOV05496.1_37
AB897757.1_prot_BAQ02840.1_60
AB897757.1_prot_BAQ02837.1_57
AB897757.1_prot_BAQ02836.1_56
ON146449.1_prot_UPW35138.1_1
MH844531.1_prot_AYP28213.1_32
AB897757.1_prot_BAQ02842.1_62
MT966873.1_p

In [27]:
# format the results : 
others_pred_formated = TropiGAT_functions.format_predictions(other_predictions , sep = "_prot_")
TropiGAT_functions.clean_print(other_predictions)

{'AB716666.1_prot_BAP15736.1_24': {'KL110': 0.9716,
                                   'KL123': 0.9985,
                                   'KL125': 0.8789,
                                   'KL127': 0.5852,
                                   'KL14': 0.7169,
                                   'KL145': 0.8723,
                                   'KL18': 0.578,
                                   'KL28': 0.554,
                                   'KL3': 0.9214,
                                   'KL43': 0.5237,
                                   'KL57': 0.7509,
                                   'KL7': 0.9922,
                                   'KL70': 0.8128,
                                   'KL74': 0.9655,
                                   'KL8': 0.8561,
                                   'KL9': 0.9793},
 'AB716666.1_prot_BAP15746.1_34': {'KL111': 0.5726,
                                   'KL128': 0.9025,
                                   'KL149': 0.698,
                             

***
# Write the results : 

In [15]:
predictions = [ferriol_predictions , bea_predictions , towndsend_predictions]

with open("/media/concha-eloko/Linux/PPT_clean/TropiGAT.results.tsv", "w") as outfile:
    for prediction in predictions:
        for prot in prediction:
            if prediction[prot] == "No hits" or len(prediction[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")

> Parse TropiGAT and Seqbased results :

In [25]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean"

tropigat_results = pd.read_csv(f"{path_project}/TropiGAT.results.tsv", sep = "\t", names = ["protein", "predictions_tropigat"])
seqbased_results = pd.read_csv(f"{path_project}/Seqbased_model.results.tsv", sep = "\t", names = ["protein", "predictions_seqbased"])


In [35]:
tropigat_results["protein_id"] = tropigat_results["protein"].apply(lambda x : x.split("_Dpo")[0])
seqbased_results["protein_id"] = seqbased_results["protein"].apply(lambda x : x.split("_A_")[0])

merged_df = tropigat_results.merge(seqbased_results, on='protein_id', how='inner')
merged_df["phage"] = merged_df["protein_id"].apply(lambda x : x.split("__")[0] if x.count("__")>0 else x.split("_")[0])

merged_df_sorted = merged_df.sort_values(by='phage', ascending=True)
merged_df_sorted

,protein_x,predictions_tropigat,protein_id,protein_y,predictions_seqbased,phage
120,A1a_00002,KL123:0.996 ; KL7:0.981 ; KL9:0.974 ; KL110:0....,A1a_00002,A1a_00002,No hits,A1a
105,A1a_00014,KL74:0.974 ; KL70:0.907 ; KL29:0.823 ; KL110:0...,A1a_00014,A1a_00014,KL151:0.708,A1a
106,A1b_00048,KL53:0.989 ; KL60:0.989 ; KL34:0.865 ; KL13:0....,A1b_00048,A1b_00048,KL157:0.57,A1b
98,A1b_00036,KL123:0.999 ; KL7:0.998 ; KL110:0.981 ; KL9:0....,A1b_00036,A1b_00036,No hits,A1b
129,A1c_00046,KL123:0.996 ; KL7:0.992 ; KL110:0.967 ; KL9:0....,A1c_00046,A1c_00046,No hits,A1c
...,...,...,...,...,...,...
128,S13a_00036,KL70:0.965 ; KL136:0.934 ; KL123:0.921 ; KL12:...,S13a_00036,S13a_00036,No hits,S13a
90,S13b_00058,KL47:0.992 ; KL64:0.913 ; KL28:0.882 ; KL3:0.7...,S13b_00058,S13b_00058,KL63:0.893,S13b
122,S13c_00055,KL123:0.976 ; KL145:0.972 ; KL38:0.969 ; KL136...,S13c_00055,S13c_00055,KL38:0.9,S13c
121,S13d_00057,KL14:0.99 ; KL21:0.975 ; KL13:0.917 ; KL62:0.8...,S13d_00057,S13d_00057,KL14:0.976,S13d


In [37]:
final_df = merged_df_sorted[["phage","protein_id", "predictions_seqbased", "predictions_tropigat"]]

final_df

,phage,protein_id,predictions_seqbased,predictions_tropigat
120,A1a,A1a_00002,No hits,KL123:0.996 ; KL7:0.981 ; KL9:0.974 ; KL110:0....
105,A1a,A1a_00014,KL151:0.708,KL74:0.974 ; KL70:0.907 ; KL29:0.823 ; KL110:0...
106,A1b,A1b_00048,KL157:0.57,KL53:0.989 ; KL60:0.989 ; KL34:0.865 ; KL13:0....
98,A1b,A1b_00036,No hits,KL123:0.999 ; KL7:0.998 ; KL110:0.981 ; KL9:0....
129,A1c,A1c_00046,No hits,KL123:0.996 ; KL7:0.992 ; KL110:0.967 ; KL9:0....
...,...,...,...,...
128,S13a,S13a_00036,No hits,KL70:0.965 ; KL136:0.934 ; KL123:0.921 ; KL12:...
90,S13b,S13b_00058,KL63:0.893,KL47:0.992 ; KL64:0.913 ; KL28:0.882 ; KL3:0.7...
122,S13c,S13c_00055,KL38:0.9,KL123:0.976 ; KL145:0.972 ; KL38:0.969 ; KL136...
121,S13d,S13d_00057,KL14:0.976,KL14:0.99 ; KL21:0.975 ; KL13:0.917 ; KL62:0.8...


In [38]:
final_df.to_csv(f"{path_project}/PPT_results.tsv", sep = "\t", header = True, index = False)